In [1]:
%config Completer.use_jedi = False
import sys
import warnings
warnings.filterwarnings('ignore')

import os
os.environ["CUDA_VISIBLE_DEVICES"]="2"
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

# import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt

MEMORY = 128#1*1024

# gpus = tf.config.list_physical_devices('GPU')
# try:
#     tf.config.set_logical_device_configuration(gpus[0],
#                                               [tf.config.LogicalDeviceConfiguration(memory_limit=MEMORY)])
#     logical_gpus = tf.config.list_logical_devices('GPU')
#     print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
# except RuntimeError as e:        
#     # Memory growth must be set before GPUs have been initialized
#     print(e) 
    
import pathlib
import pickle
from pathlib import Path
sys.path.append(f"{Path.home()}/morpho_repo")
sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")
from turing.utils import *
# from turing.tf_utils import *
# import turing.pinns as tu
# from turing.loss_functions import *


from pathlib import Path
sys.path.append(f"{Path.home()}/morpho_repo/turing_codebase")
from pde_solvers.cn import *

In [2]:
from local_utils import *

# Load parameters

In [100]:
import pandas as pd
df = pd.read_csv("df_network_analysis_full_topology_with_estimates.csv")
df["index"] = range(df.shape[0])

In [4]:
def load_dataset(path):
    with open(f"../{path}", "rb") as f:
        k_max, params, res = np.load(f, allow_pickle=True)
    (n_val, 
     b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
     b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
     b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val) = params
    params = {
              'D_A':0.01,
              'D_B':1.0,
              'n':n_val, 
              'b_A':b_A_val, 
              'mu_A':mu_A_val, 
              'V_A':V_A_val,
              'K_AA':K_AA_val, 
              'K_AB':K_AB_val,  
              'K_AC':K_AC_val,
              'b_B':b_B_val, 
              'mu_B':mu_B_val, 
              'V_B':V_B_val,
              'K_BA':K_BA_val, 
              'K_BC':K_BC_val,  
              'b_C':b_C_val, 
              'mu_C':mu_C_val, 
              'V_C':V_C_val,
              'K_CB':K_CB_val, 
              'K_CC':K_CC_val
             }
           
    return (params, res, k_max)

In [5]:
def act(x, K, n):
    """Activatrion"""
    return 1 / (1 + (K / (x + 1e-30)) ** n)


def inh(x, K, n):
    """Inhibition"""
    return 1 / (1 + (x / (K + 1e-30)) ** n)

A_act_AA = lambda A, K_AA: act(A, K_AA, 4)
A_inh_BA = lambda B, K_BA: inh(B, K_BA, 4)
A_prod = lambda A, K_AA, B, K_BA: A_act_BA(A, K_AA)*A_act_BA(B, K_BA)

B_act_AB = lambda A, K_AB: act(A, K_AB, 4)
B_inh_CB = lambda C, K_CB: inh(C, K_CB, 4)
B_prod = lambda A, K_AB, C, K_CB: B_act_AB(A, K_AB)*B_inh_CB(C, K_CB)

C_inh_AC = lambda A, K_AC: inh(A, K_AC, 4)
C_inh_BC = lambda B, K_BC: inh(B, K_BC, 4)
C_act_CC = lambda C, K_CC: act(C, K_CC, 4)
C_prod = lambda A, K_AC, B, K_BC, C, K_CC: C_inh_AC(A, K_AC)*C_inh_BC(B, K_BC)*C_act_CC(C, K_CC)

In [6]:
def to(arr):
    return arr.reshape(128, 128) 

In [8]:
np.set_printoptions(suppress=True)

In [7]:
from scipy.optimize import minimize
def simplex_corners(dim, L):
    """gives the corner of a simplex in a dim-dimensional space
       witk lenght L.
       
       dim: dimensions
       L: simplex lenght
    """
    e = np.eye(dim) * L
    return e

def grid(dim, L, repeat):
    dim_p_1 = dim + 1
    inits = np.zeros((1, dim))
    #bounds = [((0, L * 2),) *dim]
    for i in range(repeat):
        coners = simplex_corners(dim, L * (i+1))
        centers = np.ones((1, dim))*(L * (i+1))/dim
        inits = np.vstack([inits, coners, centers])
        
        #bounds.append([((L * i, L * (i+2)),) *dim] * (dim+1))
        
    return inits#, bounds
        
    
def search_single(dim, L, repeat, func, method='Powell'):
    inits = grid(dim, L, repeat)
    bounds = ((0, L * 2),) *dim
    results = []
    for i, x0 in enumerate(inits):  
        results.append(minimize(func, x0=x0, method=method, bounds=bounds))
        if i % (dim+1) == 0:
            bounds = ((0, L * (i+2)),) *dim
    
    index = np.argmin([r["fun"] for r in results])
    return results, results[index], index

def minimize_by_search(dim, L, repeat, func, options={'ftol':1e-10}):
    results, res, index = search_single(dim, L, repeat, func)
    bounds = ((0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf))
    res2= minimize(func, x0=res.x, method='L-BFGS-B', bounds=bounds, options=options)
    if res["fun"] > res2["fun"]:
        return res2.x, res, res2, results, index
    else:
        return res.x, res, res2, results, index

In [8]:
def Create_f_C(A, B, C, n, mu_C):
    def act(x, km, n=4):
        return 1 / (1 + (km / (x + 1e-20)) ** (n))

    def inh(x, km, n=4):
        return 1 / (1 + (x / (km + 1e-20)) ** (n))

    def f_c(b_C, V_C, K_AC, K_BC, K_CC):
        return b_C + V_C*inh(A, K_AC, n)*inh(B, K_BC, n)*act(C, K_CC, n) - mu_C * C
    return f_c

def minimisation_sims(index):
    print("######################################################")
    path = df["path"].iloc[index]
    print(index, path)
    (params, res, k_max) = load_dataset(path)
    
    A = to(res[-1, 0, :])
    B = to(res[-1, 1, :])
    C = to(res[-1, 2, :])
    plot_n_im([C, C_prod(A, params["K_AC"], B, params["K_BC"], C, params["K_CC"])], 
              ["C", "C activation"],          
              figsize=(12, 3)); plt.tight_layout();plt.show()


    f_c_func = Create_f_C(A, B, C, 4, params["mu_C"])
    f_c_loss = lambda args: np.sum(f_c_func(*args)**2)
    print("b_C", "V_C", "K_AC", "K_BC", "K_CC")
    print(params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"])
    orig_x = (params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"])
    x, search, search2, searchs, index = minimize_by_search(5, 5, 20, f_c_loss)
    print(x)
    return (x, orig_x, params, search, search2, searchs, index)
    

In [9]:
def to(arr):
    return arr.reshape((128, 128))

In [10]:
def get_df_param(index):
    (n_val,
     b_A_val, mu_A_val, V_A_val, K_AA_val, K_AB_val, K_AC_val,
     b_B_val, mu_B_val, V_B_val, K_BA_val, K_BC_val,
     b_C_val, mu_C_val, V_C_val, K_CB_val, K_CC_val) = df.iloc[index]["params"]
    params = {
              'D_A':0.01,
              'D_B':1.0,
              'n':n_val, 
              'b_A':b_A_val, 
              'mu_A':mu_A_val, 
              'V_A':V_A_val,
              'K_AA':K_AA_val, 
              'K_AB':K_AB_val,  
              'K_AC':K_AC_val,
              'b_B':b_B_val, 
              'mu_B':mu_B_val, 
              'V_B':V_B_val,
              'K_BA':K_BA_val, 
              'K_BC':K_BC_val,  
              'b_C':b_C_val, 
              'mu_C':mu_C_val, 
              'V_C':V_C_val,
              'K_CB':K_CB_val, 
              'K_CC':K_CC_val
             }
           
    return params

In [101]:
df[df["bad_estimates"] == 0][df.columns[-20:-10]]

,b_C,V_C,K_AC,K_BC,K_CC,b_C_est,V_C_est,K_AC_est,K_BC_est,K_CC_est
1,20.0,100.0,5.0,5.0,5.0,20.000304,85.238961,4.997341,4.993690,4.794074
13,20.0,100.0,10.0,10.0,10.0,19.998957,111.330987,10.000937,10.019551,10.289178
18,20.0,100.0,5.0,5.0,5.0,20.000000,84.259432,4.999741,5.181077,4.962777
31,10.0,100.0,5.0,5.0,5.0,10.000005,84.468316,4.997544,5.181795,4.962992
65,20.0,100.0,1.0,1.0,1.0,19.999998,96.165000,1.000042,0.999663,0.989688
74,1.0,25.0,0.5,1.0,0.5,1.002169,25.541239,0.501689,0.992598,0.507737
75,20.0,100.0,5.0,5.0,5.0,20.000000,83.788221,4.999834,5.187251,4.961674
82,1.0,10.0,5.0,5.0,5.0,1.000034,9.846333,4.993357,5.045943,5.016364
83,1.0,25.0,5.0,10.0,5.0,1.025880,26.663751,5.420625,9.808250,6.105870


In [13]:
from scipy.optimize import minimize

def rosenbrock(x1, x2, a=1, b=100):
    return (a-x1)**2 + b*(x2 - x1**2)**2

def alienor_components(epsilon, l1, bounds, ):
    """
    
    Args:
          epsilon (float): The accuracy of the estimates.
          l1 (float): Lipschitz constant
          bounds (list of tuples):
          bounds like [a_i, b_i], for each variables separatly.
          
          
    """
    n = len(bounds) 
    assert n >= 2, "The method expects two or more vairbales."
    assert np.all([len(item) == 2 
                   for item in bounds]), "bounds must be tuple of (a_i,b_i)"
    
    alpha = epsilon/(2*l1*np.sqrt(n-1))
    alphas = np.ones(n)
    
    def get_h_i(a,b,alpha):
        def h_i(t):
            return (a-b)*np.cos(alpha*t)/2 + (a+b)/2
        return h_i
    a,b = bounds[0]
    h_i = get_h_i(a,b, 1)
    h_list = [h_i]
    for i in range(1, n):
        # [a_i, b_i]
        a,b = bounds[i-1]
        # (pi/alpha)^(i-1) (b_i - a_i)
        alphas[i] = np.pi*alphas[i-1]*(b-a)/alpha
        # h_i = (a_i - b_i)cos(alpha_i theta)/2 +  (a_i + b_i)/2
        a,b = bounds[i]
        h_i = get_h_i(a,b, alphas[i])
        h_list.append(h_i)
        
        
    # l2 or Lipschitz constant of the aliemor h functions
    l2 = np.linalg.norm([(d[1]-d[0])**2 * a**2 for d,a in zip(bounds,alphas) ])/2
    theta_max = np.pi/alphas[0]
    return alpha, alphas, l2, h_list, theta_max

def minim(epsilon, l1, bounds, func, maxiter=10,method="Nelder-Mead"):
    alpha, alphas, l2, h_list, theta_max = alienor_components(epsilon, l1, bounds)
    k = 1
    L = l1 * l2
    
    theta = theta_max/2#np.pi/(2*alphas[0])  
    bounds=[(0, theta_max)]
    
    theta_epsilon = theta
    f = lambda t: func(*[ h(t) for h in h_list])
    
    f_epsilon = f(theta_epsilon)
    F_epsilon = f_epsilon - L*theta_max/2
    def single_F_k(f_theta, theta):
        def F_k(t):
            return f_theta - L*np.abs(theta - t)
        return F_k
    
    def get_F_k_func(F_k_list):
        def F_k_func(t):
            return np.max([f_i(t) for f_i in F_k_list])
        return F_k_func
    
    def get_F_k_func_vec(f_epsilon_arr, theta_arr):
        def F_k_func(t):
            return np.max(f_epsilon_arr - L*np.abs(theta_arr - t))
        return F_k_func
            
    theta_arr = np.array([theta])
    f_epsilon_arr = np.array([f_epsilon]) 
    F_k_func = get_F_k_func_vec(f_epsilon_arr, theta_arr)
    res = minimize(F_k_func, x0=0,hess=lambda x: np.zeros((x.shape[0], x.shape[0])),
                   bounds=bounds,  method=method)    
    theta = res['x'][0]        
    f_theta  =  f(theta)
    while k < maxiter:
        if f_epsilon - F_epsilon <= epsilon/2:            
            return (k, theta_arr, f_epsilon_arr, F_epsilon, F_k_func, 
                    theta_epsilon, f_epsilon, "")
        
        if f_theta < f_epsilon:
            f_epsilon = f_theta
            theta_epsilon = theta
        
        theta_arr = np.concatenate([theta_arr, [theta]])
        f_epsilon_arr = np.concatenate([f_epsilon_arr, [f_epsilon]]) 
        F_k_func = get_F_k_func_vec(f_epsilon_arr, theta_arr)
        res = minimize(F_k_func, x0=0,hess=lambda x: np.zeros((x.shape[0], x.shape[0])),
                       bounds=bounds,  method=method)
        
        theta = res['x'][0] 
        f_theta  =  f(theta)
        F_epsilon = res['fun']
        k += 1    
    return (k, theta_arr, f_epsilon_arr, F_epsilon, F_k_func, 
                    theta_epsilon, f_epsilon, f"max iteration '{maxiter}' is reached")

def alienor_components2(epsilon, l1, bounds):
    """
    
    Args:
          epsilon (float): The accuracy of the estimates.
          l1 (float): Lipschitz constant
          bounds (list of tuples):
          bounds like [a_i, b_i], for each variables separatly.
          
          
    """
    n = len(bounds) 
    assert n >= 2, "The method expects two or more vairbales."
    assert np.all([len(item) == 2 
                   for item in bounds]), "bounds must be tuple of (a_i,b_i)"
    
    alpha = epsilon/(2*l1*np.sqrt(n-1))
    alphas = np.ones(n)
    
    def get_h_i(a,b,alpha):
        def h_i(t):
            return (a-b)*np.cos(alpha*t)/2 + (a+b)/2
        return h_i
    a,b = bounds[0]
    h_i = get_h_i(a,b,1)
    h_list = [h_i]
    for i in range(1, n):
        # [a_i, b_i]
        a,b = bounds[i]
        # alphas[i-1] (alpha/pi) / (|b_i| + |a_i|)
        alphas[i] = alpha*alphas[i-1]/(np.pi*(np.abs(b)+np.abs(a)))
        # h_i = (a_i - b_i)cos(alpha_i theta)/2 +  (a_i + b_i)/2         
        h_i = h_i = get_h_i(a,b,alphas[i])#
        h_list.append(h_i)        
        
        
    # l2 or Lipschitz constant of the aliemor h functions
    l2 = np.linalg.norm([(np.abs(d[1])+np.abs(d[0]))**2 * a**2 for d,a in zip(bounds,alphas) ])/2
    #
    theta_max = np.pi/alphas[-1]    
    return alpha, alphas, l2, h_list, theta_max

def minim_2(epsilon, l1, bounds, func, maxiter=10000):
    alpha, alphas, l2, h_list, theta_max = alienor_components2(epsilon, l1, bounds)
    k = 1
    L = l1 * l2
    theta = epsilon / L    
    theta_epsilon = theta
    
    f = lambda t: func(*[ h(t) for h in h_list])
    f_epsilon = f_theta = f(theta_epsilon)
    
    while k < maxiter:
        if theta > np.pi/alphas[-1]:            
            return k, theta,theta_epsilon, f_epsilon, ""
        
        theta = theta + (epsilon + f_theta - f_epsilon)/ L        
        f_theta  =  f(theta)
        if f_theta < f_epsilon:
            f_epsilon = f_theta
            theta_epsilon = theta
        k += 1    
    return k, theta,theta_epsilon, f_epsilon, f"max iteration '{maxiter}' is reached"

In [103]:
df[df["index"] == 1][df.columns[-20:-10]]

,b_C,V_C,K_AC,K_BC,K_CC,b_C_est,V_C_est,K_AC_est,K_BC_est,K_CC_est
1,20.0,100.0,5.0,5.0,5.0,20.000304,85.238961,4.997341,4.99369,4.794074


In [206]:
# b_C V_C K_AC K_BC K_CC
# 20.0 100.0 5.0 5.0 5.0

In [15]:
run_index = 1#spots[0]
path = df[df["index"] == run_index]["path"][run_index]
print(run_index, path, df["bad_estimates"].iloc[run_index])
(params, res, k_max) = load_dataset(path)
A = to(res[-1, 0, :])
B = to(res[-1, 1, :])
C = to(res[-1, 2, :])

def Create_f_C(A, B, C, n, mu_C):
    def act(x, km, n=4):
        return 1 / (1 + (km / (x + 1e-20)) ** (n))

    def inh(x, km, n=4):
        return 1 / (1 + (x / (km + 1e-20)) ** (n))
    A_flat = A.flatten()[np.newaxis, :]
    B_flat = B.flatten()[np.newaxis, :]
    C_flat = C.flatten()[np.newaxis, :]
    def L_2_f_c(args):
        b_C, V_C, K_AC, K_BC, K_CC = args[:, 0:1], args[:, 1:2], args[:, 2:3], args[:, 3:4], args[:, 4:5]
        #b_C, V_C, K_AC, K_BC, K_CC = args[0], args[1], args[2], args[3], args[4]
        f = b_C + V_C*inh(A_flat, K_AC, n)*inh(B_flat, K_BC, n)*act(C_flat, K_CC, n) - mu_C * C_flat
        return np.sum(f**2, axis=1)
    return L_2_f_c


def Create_f_C_1_D(A, B, C, n, mu_C):
    def act(x, km, n=4):
        return 1 / (1 + (km / (x + 1e-20)) ** (n))

    def inh(x, km, n=4):
        return 1 / (1 + (x / (km + 1e-20)) ** (n))
    A_flat = A.flatten()
    B_flat = B.flatten()
    C_flat = C.flatten()
    def L_2_f_c(args):        
        b_C, V_C, K_AC, K_BC, K_CC = args[0], args[1], args[2], args[3], args[4]
        f = b_C + V_C*inh(A_flat, K_AC, n)*inh(B_flat, K_BC, n)*act(C_flat, K_CC, n) - mu_C * C_flat
        return np.sum(f**2)
    return L_2_f_c


f_c_loss = Create_f_C(A, B, C, 4, params["mu_C"])
f_c_loss_1_D = Create_f_C_1_D(A, B, C, 4, params["mu_C"])
#f_c_loss = lambda b_C, V_C, K_AC, K_BC, K_CC: np.sum(f_c_func(b_C, V_C, K_AC, K_BC, K_CC)**2)
#f_c_loss = lambda args: np.sum(f_c_func(args)**2, axis=1)
print("b_C", "V_C", "K_AC", "K_BC", "K_CC")
print(params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"])
orig_x = np.array([[params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"]]])
print("original loss:", 
      f_c_loss(
          np.array([[params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"]],
                    [params["b_C"], params["V_C"], params["K_AC"], params["K_BC"], params["K_CC"]]]))
     )

1 outputs_second_search/solution_10_0_26.npy 0
b_C V_C K_AC K_BC K_CC
20.0 100.0 5.0 5.0 5.0
original loss: [0.00164736 0.00164736]


In [64]:
bounds = [(0,30), (0,200), (0,30), (0,30), (0,30)]
epsilon = 1e-3
l1 = 1
alpha, alphas, l2, h_list2, theta_max2 = alienor_components2(epsilon, l1, bounds)
L = l1*l2
print("delta theta:", epsilon/L)
print("delta theta:", L/epsilon)


delta theta: 2.222222222222222e-06
delta theta: 450000.0


In [65]:
batch_size = 10000
L/epsilon/batch_size

45.0

In [128]:
# k2,theta2, theta_epsilon2, f_epsilon2, msg =  minim_2(
#     epsilon, l1, bounds,
#     lambda b_C, V_C, K_AC, K_BC, K_CC: f_c_loss_1_D((b_C, V_C, K_AC, K_BC, K_CC)), 
#     100)
# print(k2,theta2, theta_epsilon2, f_epsilon2, msg)

100 4.999999889358889e-05 4.999999889358889e-05 7244566.166093966 max iteration '100' is reached


In [121]:
# (h_list2[0](theta2), h_list2[1](theta2),
#                              h_list2[2](theta2), h_list2[3](theta2),
#                              h_list2[4](theta2))

(0.12497395491759278, 1.9753088054130785e-12, 0.0, 0.0, 0.0)

In [84]:
# thetas = np.linspace(0, theta_max2,  int(L/epsilon))
# params_by_theta = np.stack([h_list2[0](thetas), h_list2[1](thetas),
#                             h_list2[2](thetas), h_list2[3](thetas),
#                             h_list2[4](thetas)]).T
# theta_n = params_by_theta.shape[0]
# print(theta_n)
# batch_size = 10000
# f_thetas = np.concatenate(
#     [f_c_loss(params_by_theta[i*batch_size:(i+1)*batch_size, :]) for i in range(batch_size//theta_n + 1)])

125000


In [66]:
thetas = np.linspace(0, theta_max2,  int(L/epsilon))
params_by_theta = np.stack([h_list2[0](thetas), h_list2[1](thetas),
                            h_list2[2](thetas), h_list2[3](thetas),
                            h_list2[4](thetas)]).T

theta_n = params_by_theta.shape[0]
theta_m = params_by_theta.shape[1]

In [67]:
params_by_theta.shape, params_by_theta.nbytes

((450000, 5), 18000000)

In [68]:
from multiprocessing import Pool, shared_memory


shm = shared_memory.SharedMemory(name="params_by_theta",create=True, size=params_by_theta.nbytes)
shared_thetas = np.ndarray((theta_n,theta_m), dtype=np.float64,
                            buffer=shm.buf)

shared_thetas[:,:] = params_by_theta[:,:]


f_thetas = np.zeros(theta_n)
shm_out = shared_memory.SharedMemory(name="outputs",create=True, size=f_thetas.nbytes)
shared_outputs = np.ndarray((theta_n), dtype=np.float64,
                             buffer=shm_out.buf)




In [69]:


def sing(batch_id):
    params_shm = shared_memory.SharedMemory(name="params_by_theta")
    output_shm = shared_memory.SharedMemory(name="outputs")
    thetas = np.ndarray((theta_n,theta_m), dtype=np.float64, buffer=params_shm.buf)
    f_thetas = np.ndarray((theta_n), dtype=np.float64, buffer=output_shm.buf)
    #return params_by_theta[batch_id*batch_size:(batch_id+1)*batch_size, :]
    #return f_c_loss(thetas[batch_id*batch_size:(batch_id+1)*batch_size, :])
    f_thetas[batch_id*batch_size:(batch_id+1)*batch_size] = f_c_loss(thetas[batch_id*batch_size:(batch_id+1)*batch_size, :])
    return batch_id

batch_ids = range(int(L/epsilon/batch_size) + 1)
with Pool(55) as pool:
    res = pool.map(sing, batch_ids)

f_thetas[:] = shared_outputs[:]
#f_thetas = np.concatenate([f for f in res ]) 
shm.close()
shm.unlink()

shm_out.close()
shm_out.unlink()

In [19]:
# with open(f"f_thetas_index_1_l1_1_epsilon_minus_5.npy", "wb") as f:
#     np.save(f, f_thetas)

# with open(f"f_thetas_index_1_l1_1_epsilon_minus_5.npy", "rb") as f:
#     f_thetas = np.load(f)


In [70]:
f_thetas[10]

33543854.034744933

In [71]:
def filter_top_n(f_thetas, thetas, h_list2, n, max_dist):
    # Select and sort the top 10n f_thetas
    top_n_indices = np.argpartition(-f_thetas, -n*n)[-n*n:]
    top_n_indices = top_n_indices[np.argsort(f_thetas[top_n_indices])]
    # Convert the thetas to parameter in param space
    param_points = np.array([(h_list2[0](thetas[i]), h_list2[1](thetas[i]),
                              h_list2[2](thetas[i]), h_list2[3](thetas[i]),
                              h_list2[4](thetas[i])) for i in top_n_indices])
    # Select the corresponding top thetas
    theta_points = np.array([thetas[i] for i in top_n_indices])
    
    
    # Store the top parama points and its theta
    tops = [param_points[0]]
    tops_theta = [theta_points[0]]
    top_indices = [top_n_indices[0]]
    # Remove the top from the list
    param_points = param_points[1:]
    theta_points = theta_points[1:]
    top_n_indices = top_n_indices[1:]
    
    for i in range(n-1):
        # Find the distance between the last selected param point in
        # the top list and all the remaining        
        distances =np.linalg.norm(tops[-1] - param_points, axis=1)
        #distances = distances[np.argsort(distances)]
        # remove points that there distance to the latest one is smaller than the treshold        
        param_points = np.array([p for j,p in enumerate(param_points) if  distances[j] > max_dist])
        theta_points = np.array([t for j,t in enumerate(theta_points) if  distances[j] > max_dist])
        top_n_indices = np.array([ind for j,ind in enumerate(top_n_indices) if  distances[j] > max_dist])
        # Make sure there is any point left
        if len(theta_points) == 0:
            break
        # Store the top parama points and its theta
        tops.append(param_points[0])
        tops_theta.append(theta_points[0]) 
        top_indices.append(top_n_indices[0])
        # Remove the top from the list        
        param_points = param_points[1:]
        theta_points = theta_points[1:]
        top_n_indices = top_n_indices[1:]
        
    return (tops, tops_theta, top_indices)

In [72]:
tops, tops_theta, top_indices = filter_top_n(f_thetas, thetas, h_list2, n=20, max_dist=50)

In [73]:
for i, init_par, theta_star in zip(top_indices, tops, tops_theta):
    print(f_thetas[i])
    print(init_par)
    bounds = ((0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf))
    res3= minimize(f_c_loss_1_D, x0=init_par, method='L-BFGS-B', bounds=bounds)#, options={'ftol':1e-10})
    print(res3['fun'])
    print("success:", res3['success']," - ", res3['message'])
    print([item for item in res3['x']])
    print("="*40)

18.45911737356546
[15.66043919 87.0549918  29.51032802 15.63876056  4.12957106]
0.006506598796412541
success: True  -  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
[12.897586982663901, 22.91261374528085, 41.356265120710844, 42.89608310658578, 2.441484343994281]
27.18974877075491
[ 8.02056397 19.41679936 22.20612415 16.96541257  1.74210237]
0.0005944837514533861
success: True  -  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
[12.558775067684941, 22.331792370630673, 85.56478011909928, 49.80390496350864, 2.378298463644216]
77.11045451276563
[ 17.5541906  199.99087706  23.74941845   9.92214645   5.6103545 ]
0.9605063886056722
success: True  -  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
[15.669083584901522, 199.9962057169403, 21.127903929537474, 227.24966621209373, 5.163929502768233]
425.14827127949667
[ 14.42659787 140.69218554  29.02246677  24.62598211   4.44202918]
0.00039041912313229215
success: True  -  CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH
[12.607044226744348, 22.4

In [74]:
top_n = np.argpartition(-f_thetas, -20)[-20:]
top_n = top_n[np.argsort(f_thetas[top_n])]

In [75]:
f_thetas[top_n], thetas[top_n]

(array([ 18.45911737,  27.18974877,  34.09058336,  77.11045451,
         83.92984043, 101.21295199, 118.24783635, 123.51303291,
        167.54176129, 176.77789428, 182.31466486, 190.98690566,
        220.75672293, 229.02360527, 234.42349132, 239.08730459,
        241.41699739, 249.61381266, 251.22829009, 257.41029103]),
 array([1.02367837e+23, 6.55443329e+22, 1.07872090e+23, 1.20440215e+23,
        1.13300196e+23, 1.21055977e+23, 1.12723918e+23, 1.12734260e+23,
        6.33783550e+22, 8.39410433e+21, 1.02357496e+23, 1.10039008e+23,
        6.74226418e+22, 8.05303803e+22, 9.92335616e+22, 1.14699997e+23,
        2.88815851e+22, 1.26496304e+23, 4.48819945e+22, 1.03636024e+23]))

In [85]:
shift = 0
n = 20
init_params = np.zeros((n, 5))
final_params = np.zeros((n, 5))
init_loss = np.zeros(n)
final_loss = np.zeros(n)
for shift in range(n):

    theta_star = thetas[top_n][0 + shift]
    init_par = (h_list2[0](theta_star), h_list2[1](theta_star),
                h_list2[2](theta_star), h_list2[3](theta_star),
                h_list2[4](theta_star))
    init_params[shift, :] = init_par
    init_loss[shift] = f_thetas[top_n][0 + shift]
    #print(f_thetas[top_n][0 + shift])
    #print([item for item in init_par])
    #bounds = ((0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf))
    bounds = ((0, 200), (0, 200), (0, 200), (0, 200), (0, 200))
    res3= minimize(f_c_loss_1_D, x0=init_par, method='L-BFGS-B', bounds=bounds)#, options={'ftol':1e-10})
    #print(res3['fun'])
    #print([item for item in res3['x']])
    #print("="*40)
    final_params[shift, :] = res3['x']
    final_loss[shift] = res3['fun']

In [89]:
sorted_loss_ind = np.argsort(final_loss)
init_params = init_params[sorted_loss_ind.tolist()]
init_loss = init_loss[sorted_loss_ind.tolist()]
final_params = final_params[sorted_loss_ind.tolist()]
final_loss = final_loss[sorted_loss_ind.tolist()]

In [94]:
mutual_dist = np.linalg.norm( final_params[:-1] - final_params[1:], axis=1)

In [96]:
for i, (a,b,c,d) in enumerate(zip(final_loss, final_params,init_loss, init_params)):
    if i == n-1:
        break
    print(i, a, mutual_dist[i] , b)

0 0.00039438600947692016 131.1948107322266 [ 12.60494496  22.43245153  59.71083436 200.           2.3880023 ]
1 0.00048413830982808154 25.061623411000216 [12.63366369 22.46806771 54.87183685 68.89446865  2.39273563]
2 0.0004852691457786439 30.42975425505579 [12.57711845 22.36253152 72.35559689 50.93928427  2.38170762]
3 0.000542792208587098 35.23237324722564 [12.65135706 22.49966607 53.1598712  74.55007703  2.39610547]
4 0.0005457427432229676 121.49286955247662 [12.56650779 22.34437445 78.51155644 50.0841702   2.37971744]
5 0.0007319955829692072 211.26920045508965 [ 12.53819408  22.3042903  200.          51.12002783   2.37480733]
6 0.0008053917013258532 215.01764811864606 [ 12.68450478  22.56238488  50.10223432 200.           2.4025781 ]
7 0.001464850913085288 4.35387778386238 [ 20.00022236 100.74793581   4.99869605   5.0000997    5.00999504]
8 0.0014794276677454353 208.9939309419037 [20.0002425  96.39444195  4.99836282  4.9985101   4.95219951]
9 0.09581533329295056 29.35167584855968 [

In [ ]:
20.000304	85.238961	4.997341	4.993690	4.794074

In [76]:
bounds = ((0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf), (0, np.inf))
res3= minimize(f_c_loss_1_D, x0=init_par, method='L-BFGS-B', bounds=bounds, options={'ftol':1e-10})
print(res3)
print([item*100 for item in res3['x']])

      fun: 3.0197286137964054
 hess_inv: <5x5 LbfgsInvHessProduct with dtype=float64>
      jac: array([ 3.16090558e+00,  1.76204163e-01,  9.23261473e-05,  9.94759836e-05,
       -1.01747447e+01])
  message: 'CONVERGENCE: REL_REDUCTION_OF_F_<=_FACTR*EPSMCH'
     nfev: 198
      nit: 15
     njev: 33
   status: 0
  success: True
        x: array([0.15568844, 4.00909881, 3.20805827, 1.70596447, 0.06139482])
[15.568844153689227, 400.90988105635245, 320.80582661352753, 170.5964468243618, 6.139482385404916]


In [274]:
params_by_theta.shape

(12500000, 5)

In [127]:
bounds = [(0,1), (0,1), (0,1), (0,1), (0,1)]
epsilon = 1e-1
l1 = 1
alpha, alphas, l2, h_list, theta_max = alienor_components(epsilon, l1, bounds)
L = l1*l2
print("delta theta:", epsilon/L)
print("delta theta:", L/epsilon)


delta theta: 3.2162595044610027e-18
delta theta: 3.1092018495801856e+17
